In [1]:
import torch
import numpy as np
import pandas as pd
import utils.constants as const

from utils.data_loader import get_thresh_data_loader
from utils.xception import Mos_Xception
from openmax.openmax import OpenMax
from utils.data_split import get_species_map

/home/bit/miniforge3/envs/pace-up/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.22 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
device = torch.device('cuda:0')

In [3]:
species_map = get_species_map()
species_list = species_map.values()

In [4]:
thresh_list = []

In [5]:
for fold in range(1, 6):
    model_path = f'model_weights/fold_{fold}/model.pth'
    data_csv_path = f'data/splits/data_fold_{fold}.csv'
    weibull_path = f'weibull_models/fold_{fold}/weibull_model.pkl'

    df = pd.read_csv(data_csv_path)
    test_dl = get_thresh_data_loader(df)

    net = Mos_Xception(const.NUM_CLASSES)
    net.load_state_dict(torch.load(model_path))
    net.to(device)
    net.eval()

    om_wraper = OpenMax(net, weibull_path)
    thresh = om_wraper.get_thresh(test_dl)
    thresh_list.append(thresh)

/home/bit/miniforge3/envs/pace-up/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1716905971190/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,


In [6]:
thresh_df = pd.DataFrame(data=thresh_list, columns=['Threshold'])
thresh_df.to_csv('results/om_thresh.csv', index=False)